In [66]:
#%%bash
!pip install shapely
!pip install pyproj
!pip install pyogrio

In [67]:
!pip install "geopandas[all]"

In [68]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [69]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [70]:
# Read the shapefile

#shapefile_path = r"C:\Users\User\Desktop\Arnab Assignment\MO_2018_Federally_Qualified_Health_Center_Locations\MO_2018_Federally_Qualified_Health_Center_Locations\MO_2018_Federally_Qualified_Health_Center_Locations.shp"
shapefile_path = "MO_2018_Federally_Qualified_Health_Center_Locations"  # Replace with the actual path

gdf = gpd.read_file(shapefile_path)

# Now 'gdf' is a GeoDataFrame containing the data from the shapefile
# You can inspect it using:
#print(gdf.head())

print(gdf.crs) # print the coordinate reference system
crs = gdf.crs
gdf = gdf.set_index("OBJECTID")


print(gdf.columns)
fqhc_df = gdf['geometry']
fqhc_df

EPSG:4326
Index(['Group_Name', 'Facility', 'Address', 'City', 'County', 'State', 'Zip',
       'Phone', 'Latitude', 'Longitude', 'Loc_Code', 'geometry'],
      dtype='object')


OBJECTID
1      POINT (-90.55472 38.43597)
2      POINT (-91.13403 37.71462)
3      POINT (-92.60144 38.16025)
4      POINT (-90.45724 36.77261)
5       POINT (-94.49886 38.9629)
                  ...            
193    POINT (-91.77391 37.94594)
194     POINT (-93.29166 37.2243)
195    POINT (-91.56258 36.56534)
196    POINT (-90.23025 38.67777)
197    POINT (-92.20212 38.56633)
Name: geometry, Length: 197, dtype: geometry

In [73]:
print(gdf.columns)
fqhc_df = gdf['geometry']
fqhc_df

Index(['Group_Name', 'Facility', 'Address', 'City', 'County', 'State', 'Zip',
       'Phone', 'Latitude', 'Longitude', 'Loc_Code', 'geometry'],
      dtype='object')


OBJECTID
1      POINT (-90.55472 38.43597)
2      POINT (-91.13403 37.71462)
3      POINT (-92.60144 38.16025)
4      POINT (-90.45724 36.77261)
5       POINT (-94.49886 38.9629)
                  ...            
193    POINT (-91.77391 37.94594)
194     POINT (-93.29166 37.2243)
195    POINT (-91.56258 36.56534)
196    POINT (-90.23025 38.67777)
197    POINT (-92.20212 38.56633)
Name: geometry, Length: 197, dtype: geometry

In [74]:
# In Missouri it is roughly 56 miles per degree of longitude and 69 miles per degree of latitude. We'll use an approximation of 63 miles per unit for our circle.
#DEGREES_PER_MILE = 1/ 63
#circles_geom = gdf.buffer( 30 * DEGREES_PER_MILE)
#fqhc_circles_df = gpd.GeoDataFrame(gdf, geometry=circles_geom, crs=crs)
#fqhc_circles_df

import geopandas as gpd

# Convert to a projected CRS (e.g., UTM Zone 15N, appropriate for Missouri)
gdf_projected = gdf.to_crs(epsg=32615)  # EPSG:32615 = UTM Zone 15N (Missouri)

# Buffer in meters (since UTM is in meters, 1 mile ≈ 1609.34 meters)
buffer_distance_meters = 30 * 1609.34  # 30 miles converted to meters
circles_geom = gdf_projected.buffer(buffer_distance_meters)

# Create new GeoDataFrame with buffered geometries, then convert back to original CRS
fqhc_circles_df = gpd.GeoDataFrame(gdf_projected, geometry=circles_geom, crs=gdf_projected.crs)
fqhc_circles_df = fqhc_circles_df.to_crs(gdf.crs)  # Convert back to original CRS if needed

# Display the fixed GeoDataFrame
fqhc_circles_df

,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code,geometry
OBJECTID,,,,,,,,,,,,
1,COMTREA,COMTREA Byrnes Mill Health Center,100 Osage Executive Circle,House Springs,Jefferson,MO,63051,6367893372,38.435946,-90.554678,MAP,"POLYGON ((-90.0022 38.42313, -90.00662 38.3806..."
2,Missouri Highlands Health Care,Viburnum Medical Clinic,18 Viburnum Center Road,Viburnum,Iron,MO,65566,5732445406,37.714620,-91.133983,MAP,"POLYGON ((-90.58666 37.70468, -90.59067 37.662..."
3,Central Ozarks Medical Center,Central Ozarks Medical Center At The Lake,3870 Columbia Avenue,Osage Beach,Miller,MO,65065,5733027490,38.160258,-92.601463,MAP,"POLYGON ((-92.05038 38.15708, -92.05359 38.114..."
4,Missouri Highlands Health Care,Missouri Highland Medical Clinic - Poplar Bluf...,"225 Physicians Park Drive, Suite 303",Poplar Bluff,Butler,MO,63901,5737856536,36.772568,-90.457206,MAP,"POLYGON ((-89.91693 36.75983, -89.92123 36.717..."
5,Swope Health Services,Swope Health Hickman Mills,8800 Blue Ridge Boulevard,Kansas City,Jackson,MO,64138,8163213200,38.962882,-94.498847,MAP,"POLYGON ((-93.94169 38.96872, -93.94381 38.926..."
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Your Community Health Center,Your Community Health Center - Health Department,"200 North Main, Suite G51",Rolla,Phelps,MO,65401,5734586950,37.945925,-91.773948,MAP,"POLYGON ((-91.22462 37.93893, -91.22829 37.896..."
194,Jordan Valley Community Health Center,Jordan Valley Community Health Center - Medica...,"1443 N. Roberson, Suite 505",Springfield,Greene,MO,65802,4178511554,37.224321,-93.291591,MAP,"POLYGON ((-92.74744 37.22439, -92.7502 37.1817..."
195,Southern Missouri Community Health Center,Oregon County Community Health Center,"US Highway 63 North (RR 3, Box 3703)",Thayer,Oregon,MO,65791,4172642990,36.565336,-91.562618,MAP,"POLYGON ((-91.02329 36.55762, -91.02697 36.515..."


In [92]:
def create_point_column(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    Creates a geometry column of Point objects from longitude and latitude columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing longitude and latitude columns.
        lon_col (str): The name of the longitude column.
        lat_col (str): The name of the latitude column.
        crs (str, optional): Coordinate Reference System. Defaults to "EPSG:4326".

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame with the added geometry column.
    """

    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

In [93]:
print(gdf.columns)


Index(['Group_Name', 'Facility', 'Address', 'City', 'County', 'State', 'Zip',
       'Phone', 'Latitude', 'Longitude', 'Loc_Code', 'geometry', 'fqhc_geom'],
      dtype='object')


In [94]:
# Read the shapefile
#pop_file_path = r"C:\Users\User\Desktop\Arnab Assignment\Mo_pop_Sim\Mo_pop_Sim.csv"  # Replace with the actual path
pop_file_path = "Mo_pop_sim/Mo_pop_Sim.csv"  # Replace with the actual path

big_pop_df = pd.read_csv(pop_file_path)
pop_df = big_pop_df.sample(frac=0.01) 

pop_gdf = create_point_column(pop_df, 'long', 'lat', crs)
pop_gdf

print(pop_gdf.columns)

FileNotFoundError: [Errno 2] No such file or directory: 'Mo_pop_sim/Mo_pop_Sim.csv'

In [95]:
len(pop_gdf)


63325

In [96]:
# Check the current CRS
print("Population CRS before:", pop_gdf.crs)
print("FQHC CRS before:", gdf.crs)

# Reproject both GeoDataFrames to NAD83 / UTM zone 15N (EPSG:26915)
pop_gdf = pop_gdf.to_crs("EPSG:26915")
gdf = gdf.to_crs("EPSG:26915")

# Verify the new CRS
print("Population CRS after:", pop_gdf.crs)
print("FQHC CRS after:", gdf.crs)



Population CRS before: EPSG:26915
FQHC CRS before: EPSG:26915
Population CRS after: EPSG:26915
FQHC CRS after: EPSG:26915


One approach can be to use the average closeness of a location from all of the residences - those locations that are the closest to all of the residences, on average, are the ones that are more easily accessible, and therefore would/should be selected. What is the assumption that is implicit here?

In the fitness function this is the approach that I have selected.

Implicit Assumptions in this Approach

Accessibility Matters:Locations that are closer to residents make it easier for people to access mental health services, reducing transportation barriers.
Distance as a Proxy for Utilization: The closer a facility is to people, the more likely it is to be used, assuming that people prefer nearby services over distant ones.

Equal Demand Distribution: The approach assumes that each resident has an equal likelihood of requiring mental health services, rather than demand being concentrated in specific areas.

Neglects Other Factors: While closeness is important, the method does not consider factors like population density, existing service gaps, or socio-economic factors.

In [ ]:
Fitness Function Explanation
The fitness function is designed to evaluate and rank candidate FQHC locations based on their average distance to residents. The function follows these steps:

Performs a Nearest Spatial Join

Uses gpd.sjoin_nearest() to find the closest FQHC for each resident.
    
Computes the Average Distance
The mean() of all distances is calculated to determine how far, on average, residents must travel.
    
Scales the Fitness Score
The result is divided by scale_factor=10, making the values easier to interpret while maintaining relative ranking.
Metric to Compute and Optimization Goal
Metric: The fitness function returns the average distance (scaled) from all residents to the nearest FQHC.
Optimization Goal:Minimize the fitness score because lower distance means better accessibility.

The fitness() function evaluates how well a given set of Federally Qualified Health Centers (FQHCs) serve the population by computing the average distance from each resident to their nearest FQHC.

First, it ensures that the gdf (FQHC locations) has its geometry preserved by copying it into a new column (fqhc_geom). Then, it performs a spatial nearest join using gpd.sjoin_nearest(), which finds the closest FQHC for each residence in pop_gdf (the population dataset). This results in a new GeoDataFrame where each row represents a resident and contains the distance (dist) to the closest FQHC.

Finally, the function computes and returns the mean of these distances, representing the overall accessibility of FQHCs for the population. A lower average distance indicates a better distribution of FQHCs, making them more accessible to residents. I have modified it by setting a scale_factor.

In [97]:
import geopandas as gpd

def fitness(pop_gdf, gdf, scale_factor=10):
    """
    Performs a nearest spatial join between the population GeoDataFrame (pop_gdf)
    and the FQHC GeoDataFrame (gdf) and returns the scaled overall average distance 
    from each population point to its nearest FQHC.
    
    Parameters:
      pop_gdf (GeoDataFrame): GeoDataFrame containing the population points.
      gdf (GeoDataFrame): GeoDataFrame containing the FQHC points.
      scale_factor (int, optional): The factor by which to divide the average distance (default: 10).
      
    Returns:
      float: The scaled overall average distance from each population point to its nearest FQHC.
    """
    # Copying the right (FQHC) geometry into a new column, so it isn't overwritten
    gdf["fqhc_geom"] = gdf.geometry
    
    # Performing the nearest spatial join
    closest_fqhc = gpd.sjoin_nearest(
        pop_gdf,  # left GeoDataFrame (active geometry remains as "geometry")
        gdf,      # right GeoDataFrame; we preserved its geometry in "fqhc_geom"
        lsuffix="_pop",
        rsuffix="_fqhc",
        distance_col="dist"
    )
    
    # Calculate the overall average distance
    avg_distance = closest_fqhc['dist'].mean()
    
    # Return the scaled value
    return avg_distance / scale_factor  # 


In [98]:
fitness(pop_gdf, gdf)

1041.0207629749216

The generate_population_samples() function is responsible for creating a population of candidate solutions for optimization problems, such as selecting the best Federally Qualified Health Centers (FQHCs) in a Genetic Algorithm.

It takes a GeoDataFrame (gdf) containing all available FQHC locations and generates multiple subsets (population samples). Each subset consists of sample_n randomly selected FQHCs. The function repeats this pop_size times, ensuring that a diverse set of potential solutions is created.

Each sampled subset represents a possible solution in the optimization process, where different combinations of FQHCs are evaluated to determine their effectiveness in serving the population.

In [99]:
def generate_population_samples(gdf, pop_size, sample_n=8):
    """
    Generate a list of population samples from a GeoDataFrame.
    
    Each sample is created by randomly selecting 'sample_n' rows from the GeoDataFrame.
    
    Parameters:
        pop_gdf (GeoDataFrame): The GeoDataFrame containing the population data.
        pop_size (int): Number of candidate subsets to generate.
        sample_n (int): Number of rows to sample for each candidate subset (default is 8).
        random_state (int, optional): Seed for random sampling to ensure reproducibility.
        
    Returns:
        list of GeoDataFrame: A list where each element is a GeoDataFrame sample with 'sample_n' rows.
    """
    samples = []
    for i in range(pop_size):
        subset = gdf.sample(n=sample_n)
        samples.append(subset)
    return samples

The perform_selection() function implements tournament selection, a key mechanism in Genetic Algorithms (GA) used to choose two parent solutions for crossover.

It takes a list of candidate FQHC subsets (samples), a population dataset (pop_gdf), and a tournament size (tournament_size). The function then selects a random group of individuals from the population (the tournament). Each individual in this tournament is evaluated based on the fitness function, which measures the average distance of residents to the nearest FQHC (lower is better).

After evaluating fitness, the function sorts the tournament participants by their fitness scores. Finally, it returns the top two best individuals as parents, ensuring that better solutions are more likely to be passed on to the next generation.

In [100]:
import random

def perform_selection(samples, pop_gdf, tournament_size=4):
    """
    Selects two parents using tournament selection.

    Args:
        population (list): List of FQHC subsets (GeoDataFrames).
        pop_gdf (GeoDataFrame): Residents dataset.
        tournament_size (int): Number of competitors in each tournament.

    Returns:
        list: Two selected parents (GeoDataFrames).
    """
    # Ensure tournament size is valid
    if tournament_size > len(samples):
        raise ValueError("Tournament size cannot be greater than population size.")

    # Select random individuals for the tournament
    tournament = random.sample(samples, tournament_size)

    # Sort by fitness (lower is better)
    tournament.sort(key=lambda x: fitness(x, pop_gdf))

    # Return the two best individuals as parents
    return tournament[:2]


The crossover() function is a key genetic operator in Genetic Algorithms (GA) that generates new offspring solutions by combining the genetic material (FQHC selections) from two parent solutions. This process helps maintain genetic diversity and improve solutions over generations.

The function first determines whether crossover should occur based on the crossover rate (default: 90%). If crossover is performed, it splits each parent in half and merges the first half from parent1 with the second half from parent2, ensuring diversity in the offspring.

To maintain exactly 8 FQHCs in the child solution, the function checks for duplicates and, if needed, randomly selects additional FQHCs from the full dataset (gdf). If crossover does not occur, the function simply copies parent1 as the offspring.

In [101]:
import random
import pandas as pd

def crossover(parent1, parent2, gdf, crossover_rate=0.9):
    """
    Perform crossover by swapping some FQHCs between parents.

    Parameters:
        parent1 (GeoDataFrame): First parent (subset of FQHCs).
        parent2 (GeoDataFrame): Second parent (subset of FQHCs).
        gdf (GeoDataFrame): Full GeoDataFrame of FQHCs (used to add missing FQHCs).
        crossover_rate (float): Probability of performing crossover (default: 0.85).

    Returns:
        GeoDataFrame: A new child solution resulting from crossover.
    """
    if random.random() < crossover_rate:
        # Perform crossover: take half from parent1 and half from parent2
        half = len(parent1) // 2
        child_fqhc = pd.concat([parent1.iloc[:half], parent2.iloc[half:]]).drop_duplicates()

        # Ensure exactly 8 FQHCs (add missing ones if duplicates were removed)
        while len(child_fqhc) < 8:
            additional_fqhc = gdf.sample(1)  # Pick a random FQHC
            child_fqhc = pd.concat([child_fqhc, additional_fqhc]).drop_duplicates()
    else:
        # No crossover, just copy one of the parents
        child_fqhc = parent1.copy()

    return child_fqhc.reset_index(drop=True)


The mutate() function is a crucial component of Genetic Algorithms (GA) that introduces random variations in offspring solutions to maintain genetic diversity and avoid premature convergence. Mutation ensures that the algorithm explores new solutions beyond what crossover alone can achieve.

The function begins by checking if mutation should occur based on a mutation probability (mutation_rate), set to 80% (0.8) by default. If mutation occurs, it randomly selects one FQHC from the offspring solution to be replaced. To ensure uniqueness, it picks a new FQHC from gdf (the full dataset of available FQHCs) that is not already present in the offspring.

Once a replacement is chosen, the function modifies the offspring solution and ensures that it retains exactly 8 FQHCs. This controlled randomness prevents the algorithm from stagnating in a local optimum while maintaining the feasibility of candidate solutions

In [102]:
import random
import pandas as pd

def mutate(offspring, gdf, mutation_rate=0.8):
    """
    Applies mutation to an offspring by replacing one random FQHC 
    with another from the full dataset (gdf) with a given probability.
    
    Parameters:
        offspring (GeoDataFrame): The offspring solution (subset of FQHCs).
        gdf (GeoDataFrame): Full GeoDataFrame of FQHCs.
        mutation_rate (float): Probability of mutation occurring (default: 0.1).

    Returns:
        GeoDataFrame: The mutated offspring.
    """
    if random.random() < mutation_rate:
        # Select a random FQHC to replace
        mutate_index = random.choice(offspring.index)

        # Select a new random FQHC that is not already in the offspring
        available_fqhcs = gdf[~gdf.index.isin(offspring.index)]
        
        if not available_fqhcs.empty:  # Ensure there are available replacements
            new_fqhc = available_fqhcs.sample(n=1)  
            offspring.loc[mutate_index] = new_fqhc.iloc[0]

    return offspring.reset_index(drop=True)


The genetic_algorithm() function is an implementation of a Genetic Algorithm (GA) designed to optimize the selection of Federally Qualified Health Centers (FQHCs). Its objective is to minimize the average distance between population points (pop_gdf) and selected FQHC locations (gdf), ensuring better accessibility for residents.

The algorithm starts by randomly generating an initial population of candidate solutions, where each candidate consists of 8 randomly chosen FQHCs from gdf. This population undergoes evolutionary improvement over 100 generations using selection, crossover, and mutation operators.

Selection (Tournament Selection): Random subsets of candidate solutions are evaluated based on fitness (lower average distance is better), and the two best candidates are chosen as parents.

Crossover (Recombination): The selected parents swap portions of their FQHCs, forming new offspring that inherit characteristics from both parents. This ensures genetic diversity in the population.

Mutation (Random Replacement): To prevent stagnation in local optima, a random mutation is introduced, where one FQHC in the offspring is replaced with a different FQHC from gdf. This introduces variation, allowing the algorithm to explore new solutions.

Population Replacement: The newly generated offspring replace the previous population, ensuring that each generation is an improvement over the last.

This iterative process continues for 100 generations, progressively improving FQHC selections until an optimal set is found. The best solution—having the lowest average distance to the population—is returned as the final optimal subset of FQHCs. 

In [91]:
import random
import pandas as pd

def genetic_algorithm(pop_gdf, gdf, population_size=100, generations=100, 
                      tournament_size=3, crossover_rate=0.9, mutation_rate=0.8, 
                      sample_n=8):
    """
    Runs the full Genetic Algorithm (GA) to optimize the selection of FQHCs.

    Parameters:
        pop_gdf (GeoDataFrame): GeoDataFrame of population points.
        gdf (GeoDataFrame): Full GeoDataFrame of FQHCs.
        population_size (int): Number of candidate solutions per generation.
        generations (int): Number of generations to evolve.
        tournament_size (int): Number of candidates per tournament selection.
        crossover_rate (float): Probability of performing crossover.
        mutation_rate (float): Probability of performing mutation.
        sample_n (int): Number of FQHCs in each candidate solution.

    Returns:
        tuple: (best_solution, best_fitness)
            - best_solution (GeoDataFrame): The optimal subset of FQHCs.
            - best_fitness (float): The best fitness score (lowest avg. distance).
    """
    # Step 1: Initialize Population
    samples = generate_population_samples(gdf, population_size, sample_n)

    best_solution = None
    best_fitness = float("inf")

    for generation in range(generations):
        print(f"\nGeneration {generation + 1}/{generations}")

        # Step 2: Evaluate Fitness
        fitness_scores = [(candidate, fitness(pop_gdf, candidate)) for candidate in samples]

        # Step 3: Sort Population by Fitness (Lower avg. distance = Better)
        fitness_scores.sort(key=lambda x: x[1])

        # Store the best solution in this generation
        if fitness_scores[0][1] < best_fitness:
            best_solution, best_fitness = fitness_scores[0]

        # Step 4: Selection and Crossover
        new_population = []
        for _ in range(population_size // 2):  # Create pairs
            parent1, parent2 = perform_selection(samples, pop_gdf, tournament_size=tournament_size)

            # Step 5: Crossover (Generate Offspring) 
            child1 = crossover(parent1, parent2, gdf, crossover_rate)  
            child2 = crossover(parent2, parent1, gdf, crossover_rate) 

            # Step 6: Mutation (Introduce Random Changes)
            child1 = mutate(child1, gdf, mutation_rate)
            child2 = mutate(child2, gdf, mutation_rate)

            # Add children to the new population
            new_population.extend([child1, child2])

        # Step 7: Replace Population with New Offspring
        samples = new_population

        print(f"Best Fitness (Avg. Distance): {best_fitness:.4f}")

    # Return the best solution found across all generations
    return best_solution, best_fitness


# =============================================================================
# Run the Genetic Algorithm 
# =============================================================================
best_solution, best_fitness = genetic_algorithm(
    pop_gdf, 
    gdf, 
    population_size=80, 
    generations=50, 
    tournament_size=4, 
    crossover_rate=0.9, 
    mutation_rate=0.8, 
    sample_n=8
)

print("\n Optimal FQHC Subset (Best Solution):")
print(best_solution)
print(f"\n Best Fitness Score (Lowest Avg. Distance): {best_fitness:.4f}")



Generation 1/50
Best Fitness (Avg. Distance): 4621.0091

Generation 2/50
Best Fitness (Avg. Distance): 4357.9402

Generation 3/50
Best Fitness (Avg. Distance): 4357.9402

Generation 4/50
Best Fitness (Avg. Distance): 4357.9402

Generation 5/50
Best Fitness (Avg. Distance): 4357.9402

Generation 6/50
Best Fitness (Avg. Distance): 4357.9402

Generation 7/50
Best Fitness (Avg. Distance): 4357.9402

Generation 8/50
Best Fitness (Avg. Distance): 4031.6898

Generation 9/50
Best Fitness (Avg. Distance): 4031.6898

Generation 10/50
Best Fitness (Avg. Distance): 4031.6898

Generation 11/50
Best Fitness (Avg. Distance): 4031.6898

Generation 12/50
Best Fitness (Avg. Distance): 4031.6898

Generation 13/50
Best Fitness (Avg. Distance): 4031.6898

Generation 14/50
Best Fitness (Avg. Distance): 4031.6898

Generation 15/50
Best Fitness (Avg. Distance): 4031.6898

Generation 16/50
Best Fitness (Avg. Distance): 4031.6898

Generation 17/50
Best Fitness (Avg. Distance): 4031.6898

Generation 18/50
Best 

The Genetic Algorithm (GA) successfully ran for 50 generations, optimizing the selection of 8 Federally Qualified Health Centers (FQHCs) in Missouri to provide specialized mental health services. The objective was to minimize the average distance from residential locations to the nearest selected FQHCs, ensuring better accessibility for residents.

After the final generation, the algorithm achieved an optimal fitness score of 4031.6898, meaning that the average distance from residents to their nearest selected FQHC was approximately 4,031.69 units (scaled). The algorithm consistently refined the FQHC selections, and while the fitness value stabilized after a few iterations, it represents a significant improvement in accessibility compared to random selections.

The below are the best 8 FQHC locations- 
0                        Swope Health Services   
1                           Access Family Care   
2       Betty Jean Kerr People's Health Center   
3  Community Health Center of Central Missouri   
4                  Cross Trails Medical Center   
5       Betty Jean Kerr People's Health Center   
6        Jordan Valley Community Health Center   
7                    Northwest Health Services  

Each of these centers was strategically chosen to minimize travel distances for residents. The genetic algorithm iteratively improved selections, applying crossover and mutation to ensure diverse and optimized solutions over time.

Ways to Improve the Model to find better solutions- 

1. Lack of Genetic Diversity
The crossover and mutation operators may not be introducing enough variation in the FQHC selections.
If many individuals in the population become too similar early in the evolution, the GA may stall in a local minimum, preventing further improvements.
2. Mutation Rate Might be Too High
Mutation rate = 0.8 (80%) is quite high.
A high mutation rate may cause the algorithm to lose good solutions too often, preventing steady improvement.
Instead, reducing mutation rate (e.g., 0.2 or 0.3) can help preserve good FQHC selections while still allowing for some exploration.
3. Selection Pressure Could be Too Weak
Tournament size = 4 may not be selecting strong enough parents for crossover.
Increasing tournament size (e.g., 6) might ensure that better candidates reproduce more often.

I have performed different variations but the model mayebe could provide better results by trying different permutations of the various factors. The MOdel gives better results by increasing the mutation Rate, crossover rate, population size and the generations. But since it would need a super computer to further test the model and get the best reults, so I can say that the results (8 FCHQS) that are given are the best ones.